In [199]:
import os
import sys

"""Linux"""
os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")
sys.path.append(os.path.abspath(os.path.join("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")))

os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast/libs/linux-gpu")


#from ctypes import FormatError
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os,sys

import torch
torch.use_deterministic_algorithms(True)
import pytorch_lightning as pl
from pytorch_forecasting.data.encoders import TorchNormalizer
from pytorch_forecasting.metrics import SMAPE, RMSE
from torchmetrics import R2Score, SymmetricMeanAbsolutePercentageError, MeanSquaredError

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import os,sys
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf,pacf
from scipy.signal import find_peaks
import operator
import statsmodels.api as sm
from itertools import combinations
import pickle
from pytorch_forecasting import Baseline
import random
from pytorch_forecasting import DeepAR,NegativeBinomialDistributionLoss
from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from optuna.trial import TrialState
import plotly
#from deepar_RegionWise_LinuxGpu_prediction_dev import train_and_forecast
from sklearn.metrics import classification_report

"""
Set Random seed
"""

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
## additional seeding to ensure reproduciblility.
pl.seed_everything(0)

Global seed set to 0


0

Now for the round 0, we reduce dataset even more from 20% to 5%. num-epochs to a first value of 50. number of trials to a fixed number of 30.

In [200]:
dataset_size = 0.05
num_epochs = 30
num_trials = 30
logs_directory = "/home/optimusprime/Desktop/Timothy_EXP_1/Round_0_logs"

In [201]:
"""
Import pre-processed Data (tsfresh features)

response and target are the same thing
"""

### Your code
os.chdir('/home/optimusprime/Desktop/Data_Timothy')
############ Basically df = raw_feat_df <-------------------------
df = pd.read_parquet('EXP_1_combined_dd_ss_raw_feat_data_no_tsfresh.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow
0,0,2021-01-01 00:00:00,4,3.0,8.0
1,1,2021-01-01 01:00:00,4,1.0,13.0
2,2,2021-01-01 02:00:00,4,2.0,9.0
3,3,2021-01-01 03:00:00,4,3.0,5.0
4,4,2021-01-01 04:00:00,4,0.0,2.0
...,...,...,...,...,...
1637653,24808,2023-10-31 16:00:00,263,126.0,153.0
1637654,24809,2023-10-31 17:00:00,263,112.0,135.0
1637655,24810,2023-10-31 18:00:00,263,99.0,119.0
1637656,24811,2023-10-31 19:00:00,263,60.0,119.0


In [202]:
working_directory = '/home/optimusprime/Desktop/Timothy_EXP_1'
os.chdir(working_directory)

In [203]:
first_idx = df['time_idx'][0]
total_time_idx = df[df.group == 4].shape[0]
reduced_df_time_idx = [*range(first_idx, int(dataset_size*total_time_idx)+1, 1)]
df = df[df['time_idx'].isin(reduced_df_time_idx)]

In [204]:
numeric_cols = list(df.columns)
numeric_cols = [e for e in numeric_cols if e not in ('time_idx', 'datetime', 'group', 'target')]
print(numeric_cols)

['inflow']


In [205]:
#################### add date information ts ####################

# create datetime features from datetime column in raw_feat_df
df['_hour_of_day'] = df["datetime"].dt.hour.astype(str)
df['_day_of_week'] = df["datetime"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["datetime"].dt.day.astype(str)
df['_day_of_year'] = df["datetime"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["datetime"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["datetime"].dt.month.astype(str)
df['_year'] = df["datetime"].dt.year.astype(str)
#################### add date information ts ####################

hour_of_day_nunique = df['_hour_of_day'].nunique()
day_of_week_nunique = df['_day_of_week'].nunique()
day_of_month_nunique = df['_day_of_month'].nunique()
day_of_year_nunique = df['_day_of_year'].nunique()
week_of_year_nunique = df['_week_of_year'].nunique()
month_of_year_nunique = df['_month_of_year'].nunique()
year_nunique = df['_year'].nunique()

In [206]:
df

,time_idx,datetime,group,target,inflow,_hour_of_day,_day_of_week,_day_of_month,_day_of_year,_week_of_year,_month_of_year,_year
0,0,2021-01-01 00:00:00,4,3.0,8.0,0,4,1,1,53,1,2021
1,1,2021-01-01 01:00:00,4,1.0,13.0,1,4,1,1,53,1,2021
2,2,2021-01-01 02:00:00,4,2.0,9.0,2,4,1,1,53,1,2021
3,3,2021-01-01 03:00:00,4,3.0,5.0,3,4,1,1,53,1,2021
4,4,2021-01-01 04:00:00,4,0.0,2.0,4,4,1,1,53,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
1614081,1236,2021-02-21 12:00:00,263,102.0,83.0,12,6,21,52,7,2,2021
1614082,1237,2021-02-21 13:00:00,263,89.0,72.0,13,6,21,52,7,2,2021
1614083,1238,2021-02-21 14:00:00,263,72.0,84.0,14,6,21,52,7,2,2021
1614084,1239,2021-02-21 15:00:00,263,65.0,78.0,15,6,21,52,7,2,2021


In [207]:
print(hour_of_day_nunique)
print(day_of_week_nunique)
print(day_of_month_nunique)
print(day_of_year_nunique)
print(week_of_year_nunique)
print(month_of_year_nunique)
print(year_nunique)

24
7
31
52
8
2
1


In [208]:
unique_values_group_no_ts_fresh = list(df['group'].unique())
print(len(unique_values_group_no_ts_fresh), "\n")
print(unique_values_group_no_ts_fresh)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [209]:
zones_time_idx_no_ts_fresh = {}
for i in unique_values_group_no_ts_fresh:
    df1 = df[df['group'] == i]
    time_idx_list = df1['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_no_ts_fresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df1.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  4  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  12  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  13  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  24  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  41  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  42  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  43  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  45  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  48  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  50  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  68  has  1241  number of rows in df_ts_fresh 

First idx:  0 ; Last idx:  1240 

Group  74  has  1241  number of r

In [210]:
encoder_len = 8 # looks back 8 hours in the past for prediction
pred_len = 1 # 1, 6, 12 # how many timeseteps you want to predict in the future (forecasting), 1 means present (nowcasting)
#cov_lag_len= pred_len

# not using covariates yet, so put as 0
cov_lag_len= 0 #we can use forecasted values, even for inflow

####### train val test split of tsfresh dataframe #########

### you want a the idx for each train, validation and test

### train = 0.7 of each zone (eg. time idx 167 to roughly 20000) eg train = 0.7 * (24812-167)
### validation = 0.2 of each zone (eg. time idx 20000 to 22000)
### test = 0.1 of each zone (eg. time idx 22000 to 24000)

Target = 'target'

total_time_idx = df[df.group == 4].shape[0]

first_idx = df.loc[df['group'] == 4, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = total_time_idx-1
# val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
# tes_start_idx = val_stop_idx - encoder_len
# tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
# print(tes_start_idx)
# print(tes_stop_idx)

868
860
1240


In [211]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
# test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
# print(test_time_idx[0], test_time_idx[-1])

0 868
860 1240


In [212]:
# unique_zones = list(df['group'].unique())

In [213]:
"""
set inputs here
(hyperparameters grid search)

"""
######### Network Architecture ###################
# p = 10 # patience no. of epochs

Loss=NegativeBinomialDistributionLoss() # because we assume that the demand follows a negative bionmial distribution

######### Network Architecture ###################


######### Training Routine ###################
fdv_steps = 10 # fast_dev_run -> small set of training to see if training is happening correctly (one of the params in deepar model)
######### Training Routine ###################


# this is for naive forecast, getting past week's demand and predicting purely based on that
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################
season_len = 168 # length of season ()
num_past_seas = 6 # number of past seasons to use in averaging (look at 6 week's back, getting average of these 6 values)
#seas_pred_strt_idx = 2035 # seasonal naive forecast start index, in hours use the df dataframe
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################


In [214]:
"""
Full Training Routine
with hyperparmeter grid search

Load data into TimeSeriesDataSet object

for fast development run
uncomment fast_dev_run = fdv_steps

"""

# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=p, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

# param_comb_cnt=-1
#for neu,lay,bat,lr,enc_len,pred_len,drop,cov_pair,num_ep in product(*[x for x in hparams_grid.values()]):


file_path = "trial_parameters.json"
# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)

import json
def save_trial_parameters(study, trial):
    trial_params = {'trial_num':trial.number ,'trial_params' :trial.params}
    with open("trial_parameters.json", "a") as f:
        json.dump(trial_params, f)
        f.write('\n')


def objective(trial): #optuna objective function to find best hyperparameters
# minimize the average validation loss

  # optuna tries to find best combination given this hyperparameter searchspace

  # first start broad
  neu = trial.suggest_int(name="neu",low=10,high=1000,step=100,log=False) # number of neurons in lstm cell
  lay = trial.suggest_int(name="lay",low=1,high=10,step=2,log=False) # number of lstm layers
  bat = trial.suggest_int(name="bat",low=32,high=1024,step=32,log=False) # batch size
  lr = trial.suggest_float(name="lr",low=0.00001,high=0.1,log=True) #learning rate, log does not have step
  drop = trial.suggest_float(name="dropout",low=0,high=0.8,step=0.2,log=False) # dont use some of the cells to distribute learning across the cells


  num_cols_list = numeric_cols
  cat_dict = {"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day),
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year),
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)}
  cat_list = ["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
  embed_size_dict = {'_hour_of_day':(hour_of_day_nunique+1,5),'_day_of_week':(day_of_week_nunique+1,5),'_day_of_month':(day_of_month_nunique+1,5), '_day_of_year':(day_of_year_nunique+1,8), '_week_of_year':(week_of_year_nunique+1,3), '_month_of_year':(month_of_year_nunique+1,3),'_year':(year_nunique+1,1)}
  # ^ converting hour into a 5 dimensional vector, etc etc, day of year = number of unique values in this column + 1, same for week of year, month of year and year

  train_dataset = TimeSeriesDataSet(
      df[(df['time_idx'].isin(train_time_idx))],
      time_idx="time_idx",
      target=Target,
      categorical_encoders=cat_dict,
      group_ids=["group"],
      min_encoder_length=encoder_len,
      max_encoder_length=encoder_len,
      min_prediction_length=pred_len,
      max_prediction_length=pred_len,
      time_varying_unknown_reals=[Target],
      time_varying_known_reals=num_cols_list,
      time_varying_known_categoricals=cat_list,
      add_relative_time_idx=False,
      randomize_length=False,
      scalers={},
      target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None)
    )

  val_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[(df['time_idx'].isin(val_time_idx))], stop_randomization=True, predict=False)
  # test_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[df['time_idx'].isin(test_time_idx)], stop_randomization=True)

  train_dataloader = train_dataset.to_dataloader(train=True, batch_size=bat)
  val_dataloader = val_dataset.to_dataloader(train=False, batch_size=bat)
  # test_dataloader = test_dataset.to_dataloader(train=False, batch_size=bat)

  
  # trial.set_user_attr("num_ep", num_ep)

  early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=True,
    mode='min'
  )

  
  """
  Machine Learning predictions START
  1) DeepAR

  """
  trainer = pl.Trainer(
      max_epochs=num_epochs,
      # accelerator="auto",
      #gpus=[0],
      accelerator='gpu',
      devices=[0],
      gradient_clip_val=0.1,
      limit_train_batches=1.0,
      limit_val_batches=1.0,
      # fast_dev_run=fdv_steps,
      logger=True,
      callbacks=[lr_logger, early_stop_callback],    #, early_stop_callback]
      default_root_dir=logs_directory
  )


  #print(f"training routing:\n \n {trainer}")
  deepar = DeepAR.from_dataset(
      train_dataset,
      learning_rate=lr,
      hidden_size=neu,
      rnn_layers=lay,
      dropout=drop,
      loss=Loss,
      log_interval=20,
      log_val_interval=6,
      log_gradient_flow=False,
      embedding_sizes=embed_size_dict
      # reduce_on_plateau_patience=3,
  )

  #print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")
  # print(f"Model :\n \n {deepar}")
  torch.set_num_threads(10)
  trainer.fit(
      deepar,
      train_dataloaders=train_dataloader,
      val_dataloaders=val_dataloader,
  )

  val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  # val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  trial.report(val_wmape, num_epochs)
  # trial.report(val_wmape, num_ep)

  # Handle pruning based on the intermediate value.
  if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

  return val_wmape #optuna is trying to minimise this <---


######### optuna results #####################
if __name__ == "__main__":

  study = optuna.create_study(direction="minimize")
  study.optimize(objective, n_trials=num_trials,callbacks=[save_trial_parameters]) # can specify the number of trials to a number less than 960 for example, eg can set n_trials = 100

  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  print("Best trial:")
  trial = study.best_trial
  # best_num_ep = study.trials[trial.number].user_attrs['num_ep']

  print("  Value: ", trial.value)

  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
########## optuna results #####################

[I 2024-03-28 12:57:47,409] A new study created in memory with name: no-name-24d3f66d-9a74-4d23-830c-8d1ff17a140c
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
Missing logger folder: /home/optimusprime/Desktop/Timothy_EXP_1/Round_0_logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                             | Params
----------------------------------------------------------------------------
0 | loss                   | NegativeBinomialDistributionLoss | 0     
1 | logging_metrics        | ModuleList                       | 0     
2 | embeddings             | MultiEmbedding                   | 787   
3 | rnn                    | LSTM                     

Epoch 0: 100%|██████████| 848/848 [00:22<00:00, 37.35it/s, loss=4.35, v_num=0, train_loss_step=4.350, val_loss=4.490]

Metric val_loss improved. New best score: 4.486


Epoch 1: 100%|██████████| 848/848 [00:23<00:00, 36.82it/s, loss=3.92, v_num=0, train_loss_step=3.990, val_loss=4.100, train_loss_epoch=19.30]

Metric val_loss improved by 0.390 >= min_delta = 0. New best score: 4.096


...

Monitored metric val_loss did not improve in the last 5 records. Best score: 2.744. Signaling Trainer to stop.


Epoch 22: 100%|██████████| 318/318 [00:16<00:00, 18.88it/s, loss=2.59, v_num=28, train_loss_step=2.570, val_loss=2.750, train_loss_epoch=2.580]

[I 2024-03-28 15:18:34,083] Trial 28 finished with value: 33.01414108276367 and parameters: {'neu': 210, 'lay': 7, 'bat': 256, 'lr': 0.028873485190812002, 'dropout': 0.0}. Best is trial 2 with value: 32.41532897949219.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                             | Params
----------------------------------------------------------------------------
0 | loss                   | NegativeBinomialDistributionLoss | 0     
1 | logging_metrics        | ModuleList                       | 0     
2 | embeddings             | MultiEmbedding                   | 787   
3 | rnn                    | LSTM                             | 10.2 M
4 | distribution_projector | Linear                           | 1.4 K 
----------------------------------------------------------------------------
10.2 M    Trainable params
0         Non-

Epoch 0:  42%|████▏     | 1066/2545 [00:24<00:33, 44.42it/s, loss=2.82, v_num=29, train_loss_step=2.830]

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("  Best trial: ")
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# print("Best number of epochs: ", best_num_ep)

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
  Best trial: 
  Value:  59.743770599365234
  Params: 
    neu: 610
    lay: 5
    bat: 320
    lr: 0.001488627874304404
    dropout: 0.6000000000000001


In [ ]:
from IPython.display import display
fig = optuna.visualization.plot_parallel_coordinate(study)
display(fig)
fig = optuna.visualization.plot_optimization_history(study)
display(fig)
fig = optuna.visualization.plot_slice(study)
display(fig)
fig = optuna.visualization.plot_param_importances(study)
display(fig)

In [ ]:
print("  Best trial: ", trial)

  Best trial:  FrozenTrial(number=2, state=TrialState.COMPLETE, values=[59.743770599365234], datetime_start=datetime.datetime(2024, 3, 28, 12, 45, 28, 902208), datetime_complete=datetime.datetime(2024, 3, 28, 12, 45, 43, 645232), params={'neu': 610, 'lay': 5, 'bat': 320, 'lr': 0.001488627874304404, 'dropout': 0.6000000000000001}, user_attrs={}, system_attrs={}, intermediate_values={1: 59.743770599365234}, distributions={'neu': IntDistribution(high=910, log=False, low=10, step=100), 'lay': IntDistribution(high=9, log=False, low=1, step=2), 'bat': IntDistribution(high=1024, log=False, low=32, step=32), 'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'dropout': FloatDistribution(high=0.8, log=False, low=0.0, step=0.2)}, trial_id=2, value=None)
